In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("sample").master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [2]:
emp = spark.read.format("csv").option("header","true").load("C:/Users/b2en/Downloads/Spark 실습/실습문제_20210324/data/emp.csv")
amt_sal = spark.read.format("csv").option("header","true").load("C:/Users/b2en/Downloads/Spark 실습/실습문제_20210324/data/금액제급여.csv")
hobong_sal = spark.read.format("csv").option("header","true").load("C:/Users/b2en/Downloads/Spark 실습/실습문제_20210324/data/호봉제급여.csv")
yearly_sal = spark.read.format("csv").option("header","true").load("C:/Users/b2en/Downloads/Spark 실습/실습문제_20210324/data/연봉제급여.csv")
sal_pay = spark.read.format("csv").option("header","true").load("C:/Users/b2en/Downloads/Spark 실습/실습문제_20210324/data/급여지급.csv")
bonus_sal = spark.read.format("csv").option("header","true").load("C:/Users/b2en/Downloads/Spark 실습/실습문제_20210324/data/기타급여.csv")
iss_hist = spark.read.format("csv").option("header","true").load("C:/Users/b2en/Downloads/Spark 실습/실습문제_20210324/data/발령이력.csv")
dept = spark.read.format("csv").option("header","true").load("C:/Users/b2en/Downloads/Spark 실습/실습문제_20210324/data/부서.csv")
pjt = spark.read.format("csv").option("header","true").load("C:/Users/b2en/Downloads/Spark 실습/실습문제_20210324/data/프로젝트.csv")
pjt_use_hist = spark.read.format("csv").option("header","true").load("C:/Users/b2en/Downloads/Spark 실습/실습문제_20210324/data/프로젝트투입이력.csv")
vc_use_hist = spark.read.format("csv").option("header","true").load("C:/Users/b2en/Downloads/Spark 실습/실습문제_20210324/data/휴가사용이력.csv")

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.window import *

- 월별 평균 급여지급 합계를 구하시오. (단, 이미 퇴사한 직원의 급여는 계산에서 제외)
- 월별 퇴사자/입사자 수를 구하시오.
- 월별 휴가를 사용한 사원의 수를 구하시오. (사원 1명이 휴가 3일 사용한 경우에도 건수는 1로 집계)


In [4]:
retired= emp.withColumnRenamed("퇴사일자","end_date").where(expr("end_date is not null"))#퇴사자의 사원번호를 얻을 수 있는 데이터프레임

In [5]:
retired

사원번호,사원명,부서코드,직급,입사일자,end_date,급여구분,월급여,등록일시
102,한영애,30,대리,2008/05/01 00:00:00,2010/12/01 00:00:00,연봉,1540000,2008/05/01 14:12:00
110,이선희,10,차장,2000/03/01 00:00:00,2007/12/01 00:00:00,호봉,1790000,2000/03/01 14:00:00


In [7]:
with_no_retired = sal_pay.join(retired,sal_pay["사원번호"] == retired["사원번호"],"left_anti")#퇴사자가 포함되지않은 급여지급 데이터프레임

In [8]:
with_no_retired

급여월,사원번호,월급여,기본급,수당,성과급
201001,101,1400000,1300000,100000,null
201001,103,1600000,1300000,100000,200000
201001,104,1680000,1460000,220000,null
201001,105,1540000,1440000,100000,null
201001,106,1700000,1400000,100000,200000
201001,107,1770000,1550000,220000,null
201001,108,1600000,1500000,100000,null
201001,109,1470000,1350000,120000,null
201001,111,1530000,1530000,null,null
201001,112,1500000,1400000,100000,null


In [10]:
avg_sal_no_retired = with_no_retired.groupBy("급여월").agg(round(avg("월급여"),0).alias("평균급여액")) #퇴사자가 포함되지 않은 월별 평균급여액

In [24]:
retired_num = emp.withColumn("퇴사일자",translate(substring(col("퇴사일자"),1,7),'/',''))\
                 .groupBy(col("퇴사일자")).agg(count("*").alias("퇴사자수"))
#월별 퇴사자수를 구하는 데이터프레임

In [27]:
monthly_vc_use_hist = vc_use_hist.withColumn("휴가사용월",substring(col("휴가사용일자"),1,6))\
                                 .groupBy(col("휴가사용월"),col("사원번호")).agg(countDistinct("사원번호").alias("휴가사용사원수"))
#월별 휴가사용사원수를 구하는 데이터프레임

In [28]:
monthly_vc_use_hist

휴가사용월,사원번호,휴가사용사원수
201106,103,1
201011,109,1
201001,101,1
201002,109,1
201105,108,1
201011,105,1
201011,101,1
201008,102,1


In [30]:
homework2_1 = avg_sal_no_retired.join(retired_num,avg_sal_no_retired["급여월"] == retired_num["퇴사일자"],"left_outer")\
                                .join(monthly_vc_use_hist,avg_sal_no_retired["급여월"] == monthly_vc_use_hist["휴가사용월"],"left_outer")\
                                .groupBy(avg_sal_no_retired["급여월"],avg_sal_no_retired["평균급여액"],retired_num["퇴사자수"])\
                                .agg(sum(monthly_vc_use_hist["휴가사용사원수"]).alias("휴가사용사원수"))\
                                .orderBy(avg_sal_no_retired["급여월"])
#급여월별로 휴가사용한 사원수를 모두 더해서 휴가사용사원수를 구한다.

In [32]:
homework2_1

급여월,평균급여액,퇴사자수,휴가사용사원수
201001,1579000.0,null,1
201002,1579000.0,null,1
201003,1579000.0,null,null
201004,1579000.0,null,null
201005,1579000.0,null,null
201006,1579000.0,null,null
201007,1579000.0,null,null
201008,1579000.0,null,1
201009,1579000.0,null,null
201010,1579000.0,null,null


- 사원별 휴가 사용일수를 구하시오. (휴가를 사용하지 않은 사원도 포함)
- 호봉제 사원의 경우, 호봉도 포함
- 각 사원별 최초/최종 직급을 포함
- 각 사원별 최초 지급된 급여액을 포함


In [46]:
#필요한 데이터프레임 사원,부서,호봉제급여,휴가사용이력,발령이력,급여지급(emp,dept,hobong_sal,vc_use_hist,iss_hist,sal_pay)
bseq_fseq = iss_hist.withColumn("RN",row_number().over(Window.partitionBy("사원번호").orderBy("이력순번")))\
                    .withColumn("DESC_RN",row_number().over(Window.partitionBy("사원번호").orderBy(desc("이력순번"))))
#최초순번 최종순번을 포함한 데이터프레임

In [38]:
bseq_lseq

사원번호,이력순번,발령일자,직급,부서코드,RN,DESC_RN
101,2,2010/03/01 00:00:00,대리,30,2,1
101,1,2008/03/01 00:00:00,사원,30,1,2
112,1,2004/03/01 00:00:00,과장,20,1,1
107,1,2001/10/01 00:00:00,차장,30,1,1
110,1,2000/03/01 00:00:00,차장,10,1,1
104,3,2008/05/01 00:00:00,과장,10,3,1
104,2,2005/05/01 00:00:00,대리,10,2,2
104,1,2003/05/01 00:00:00,사원,10,1,3
102,1,2008/05/01 00:00:00,대리,30,1,1
111,1,2003/03/01 00:00:00,차장,30,1,1


In [47]:
brank_frank = bseq_fseq.withColumn("최초직급",when(col("RN") == 1,col("직급")))\
                       .withColumn("최종직급",when(col("DESC_RN") == 1,col("직급")))\
                       .groupBy(col("사원번호")).agg(min("최초직급").alias("최초직급"),min("최종직급").alias("최종직급"))
#sql에서는 min(case when rn = 1 then 직급 end) 인 구문을 pyspark에서는 withcolumn으로 when(col("RN") == 1,col("직급"))처럼 열을 추가시켜서
#group by.agg에서 min으로 묶어준다.

In [48]:
brank_frank

사원번호,최초직급,최종직급
101,사원,대리
112,과장,과장
107,차장,차장
110,차장,차장
104,사원,과장
102,대리,대리
111,차장,차장
103,사원,대리
108,차장,차장
106,과장,과장


In [51]:
bsal = sal_pay.groupBy(col("사원번호")).agg(min("급여월").alias("최초급여월"),min("월급여").alias("최초월급여"))
#급여지급 데이터프레임에서 사원번호별 최초급여월의 급여를 얻어내기위해 사원번호별,최초급여월,월급여를 구하는 데이터프레임

In [52]:
bsal

사원번호,최초급여월,최초월급여
101,201001,1400000
112,201001,1500000
107,201001,1770000
104,201001,1680000
102,201001,1540000
111,201001,1530000
103,201001,1600000
108,201001,1600000
106,201001,1700000
105,201001,1540000


In [60]:
vc_use_num = vc_use_hist.groupBy(col("사원번호")).agg(count("휴가사용일자").alias("휴가사용일수"))
#휴가사용이력 데이터프레임에서 사우너번호별 휴가사용일수를 구하는 데이터프레임

In [61]:
vc_use_num

사원번호,휴가사용일수
101,4
102,4
103,1
108,1
105,1
109,2


In [67]:
homework2_2 = emp.join(dept,emp["부서코드"] == dept["부서코드"],"inner")\
                 .join(hobong_sal,emp["사원번호"] == hobong_sal["사원번호"],"left_outer")\
                 .join(brank_frank,emp["사원번호"] == brank_frank["사원번호"],"inner")\
                 .join(bsal,emp["사원번호"] == bsal["사원번호"],"left_outer")\
                 .join(vc_use_num,emp["사원번호"] == vc_use_num["사원번호"],"left_outer")\
                 .select(emp["사원명"],dept["부서명"],hobong_sal["호봉"],vc_use_num["휴가사용일수"],brank_frank["최초직급"],brank_frank["최종직급"],bsal["최초월급여"])
#각 데이터프레임을 사원데이터프레임의 사원번호로 조인한다.

In [68]:
homework2_2

사원명,부서명,호봉,휴가사용일수,최초직급,최종직급,최초월급여
조동진,전산팀,null,4,사원,대리,1400000
한영애,전산팀,null,4,대리,대리,1540000
조규찬,영업팀,null,1,사원,대리,1600000
이상은,인사팀,null,null,사원,과장,1680000
조덕배,인사팀,9,1,사원,과장,1540000
장필순,인사팀,8,null,과장,과장,1700000
오석준,전산팀,null,null,차장,차장,1770000
이규석,영업팀,11,1,차장,차장,1600000
전수경,전산팀,null,2,대리,대리,1470000
이선희,인사팀,13,null,차장,차장,null
